In [ ]:
pip install selenium

     |████████████████████████████████| 954 kB 15.5 MB/s 
     |████████████████████████████████| 356 kB 82.6 MB/s 
     |████████████████████████████████| 138 kB 69.4 MB/s 
     |████████████████████████████████| 3.5 MB 53.7 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!apt-get update
!apt install chromium-chromedriver

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,210 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import csv
import urllib # 한글 - 유니코드 인코딩
import re
from selenium import webdriver
import urllib.request

In [ ]:
Author = []
Title_kor = []
Title_eng = []
Book = []
Keyword = []
Txt_kor= []
Txt_eng = []
Url = []

In [ ]:
def get_URL(page, word):
    word = urllib.parse.quote(word)
    url_before_page="http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery="+word+"&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    url_after_page="&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query="+word
    url=url_before_page+page+url_after_page
    return url

In [ ]:
def get_link(search_word, page_num):
    for i in range(page_num):
        current_page=i*10
        url=get_URL(str(current_page), search_word)
        source_code_from_URL = urllib.request.urlopen(url)
        soup=BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
        try: 
            page_link = soup.select("li > div.cont > p.title > a") 
        except:
            page_link = ""
        
        if i != 0 and i%2 == 0: 
            print("Waiting...for get URL") 
            time.sleep(10) 

        print("crawling start page {}".format(i))
        for j in range(10):
            try:
              page_url = "http://riss.or.kr"+page_link[j].attrs['href'] 
            except:
              print("Index error!")
              break
            # print(page_url)
            reference_data = get_reference(page_url)

            Author.append(reference_data[0])
            Title_kor.append(reference_data[1])
            Title_eng.append(reference_data[2])
            Book.append(reference_data[3])
            Keyword.append(reference_data[4])
            Txt_kor.append(reference_data[5])
            Txt_eng.append(reference_data[6])
            Url.append(reference_data[7])

        print("crawling done page {}".format(i))

    mydict = {'저자': Author,
              '국문제목': Title_kor,
              '영문제목': Title_eng,
              '수록지': Book,
              '핵심어': Keyword,
              '국문 요약':Txt_kor,
              '영문 요약':Txt_eng,
              '링크': Url}
    
    save_csv(mydict, search_word)

In [ ]:
def get_reference(url):
    # colab에서 사용할 때 옵션들
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless') 
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome("chromedriver", options=chrome_options) 
    driver.get(url) 
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    title = soup.find("h3",{"class":"title"})
    
    title_txt = title.get_text("", strip=True).split('=')
    
    title_kor = re.sub("\n\b", "", str(title_txt[0]).strip())
    try:
      title_eng = str(title_txt[1]).strip()
    except:
      title_eng = "None"
    
    txt_box = []
    
    for text in soup.find_all("div", {"class":"text"}):
        txt = text.get_text("", strip=True)
        txt_box.append(txt)
    
    try:
      txt_kor=txt_box[1]
    except:
      txt_kor = "None"
    try:
      txt_eng=txt_box[3]
    except:
      txt_eng = "None"
    
    author = "None"
    book = "None"
    keyword = "None"
    detail_box = []
    detail_info = soup.select("#soptionview > div > div > div.infoDetail.on > div.infoDetailL > ul > li > div > p") 
    
    for detail in detail_info:
        detail_concept=detail.get_text("", strip=True)
        
        detail_wrap=[]
        detail_wrap.append(detail_concept)
        
        detail_box.append(detail_wrap)
        
    author = ",".join(detail_box[0])
    
    book=(
        "".join(detail_box[2]+detail_box[3]).replace("\n", "").replace("\t","").replace(" ","")
        + "p."
        +"".join(detail_box[-2])
    )
    
    keyword=",".join(detail_box[6])
        
    reference_data = [
        author,
        title_kor,
        title_eng,
        book,
        keyword,
        txt_kor,
        txt_eng,
        url,
    ]

    driver.close()
    return reference_data

In [ ]:
def save_csv(mydict, search_word):
    dataframe = pd.DataFrame.from_dict(mydict, orient='index')
    dataframe = dataframe.transpose()
    dataframe.to_csv(search_word+'_paper_crawling.csv', index=False, encoding='utf-8')

In [ ]:
search_word = '블록체인'
page_num = 10
get_link(search_word, page_num)

# install selenium : https://chancoding.tistory.com/136 

crawling start page 0
crawling done page 0
crawling start page 1
crawling done page 1
Waiting...for get URL
crawling start page 2
crawling done page 2
crawling start page 3
crawling done page 3
Waiting...for get URL
crawling start page 4
crawling done page 4
crawling start page 5
crawling done page 5
Waiting...for get URL
crawling start page 6
crawling done page 6
crawling start page 7
crawling done page 7
Waiting...for get URL
crawling start page 8
crawling done page 8
crawling start page 9
crawling done page 9
